# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

1. [Шаг 1. Откройте файл с данными и изучите общую информацию ](#start)
    * [Импорт библиотек](#import)
    * [Изучение данных](#study)
2. [Шаг 2. Предобработка данных](#preprocessing)
    * [Обработка пропусков](#processing)
    * [Замена типа данных](#null)
    * [Обработка дубликатов](#duplicated)
    * [Лемматизация](#lemma)
    * [Категоризация данных](#class)
3. [Шаг 3. Ответьте на вопросы](#item)
4. [Шаг 4. Общий вывод](#output)
5. [Чек-лист готовности проекта](#list)

## Шаг 1. Откройте файл с данными и изучите общую информацию 
<a id='start'></a>

### Импорт библиотек
<a id='import'></a>

In [1]:
import pandas as pd # импорт библиотеки pandas

from pymystem3 import Mystem # импортируем pymystem3
from collections import Counter # вызываем специальный контейнер Counter из модуля collections
from IPython.display import display #импортируем метод display из библиотеки IPython.display

### Изучение данных
<a id='study'></a>

Прочитаем файл data.csv и сохраним данные в переменной data.
Получение первых 20 строк таблицы

In [2]:
data = pd.read_csv('/datasets/data.csv') # чтение файла с данными с сохранением в data
data.head(20) # получение первых 20 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Посмотрим общую информацию о датафрейме.

In [3]:
data.info() # получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.describe() #просмотр некоторых основных статистических данных

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
data.isnull().sum() # определение количества NaN элементов в записях (Ожидаемые)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
data.nunique() # количество уникальных значений для каждого столбца .

children                8
days_employed       19351
dob_years              58
education              15
education_id            5
family_status           5
family_status_id        5
gender                  3
income_type             8
debt                    2
total_income        19351
purpose                38
dtype: int64

In [7]:
data.isna().sum() # определение количества NaN элементов в записях (Странные)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

 **Вывод**
    
 Всего датафрейм data содержит 12 столбцов: 
   * children — количество детей в семье
   * days_employed — общий трудовой стаж в днях
   * dob_years — возраст клиента в годах
   * education — уровень образования клиента
   * education_id — идентификатор уровня образования
   * family_status — семейное положение
   * family_status_id — идентификатор семейного положения
   * gender — пол клиента
   * income_type — тип занятости
   * debt — имел ли задолженность по возврату кредитов
   * total_income — ежемесячный доход
   * purpose — цель получения кредита
 
Датафрейм содержит 12 признаков и 21525 объекта, есть пропуски. Типы данных столбцов:  float64(2), int64(5), object(5). Память занимает 2.0+ MB 

При этом в столбцах: days_employed, total_income по 2174 NaN объектов, в остальных стобцах объекты значимы - не  NaN.
   * children - выявлены некорректные данные: максимальное и минимальное количество детей - выбросами. 
   * days_employed - встречаются отрицательные значения.
   * dob_years - возраст равен нулю.
   * education - заполнение прописными и сточными буквами.
    
Проблемы, которые нужно решать - пропуски, дубликаты. 
  
Для проверки рабочих гипотез особенно ценны столбцы сhildren, education и total_income.  Данные из столбца purpose позволят узнать цели кредита.


## Шаг 2. Предобработка данных
<a id='preprocessing'></a>

### Обработка пропусков
<a id='processing'></a>

Для столбца days_employed посчитаем среднее значение. 

In [8]:
days_employed_mean = data['days_employed'].mean() #Посчитаем среднее значение. 

Проведем замену в этом столбце пропущенных значений на рассчитаное среднее значение по столбцу days_employed.

In [9]:
data['days_employed'] = data['days_employed'].fillna(value=days_employed_mean) #Заменим пропущенные значения на среднее значение. 

Выведем первые 20 строк таблицы на экран.

In [10]:

data.head(20) #Выведим на экран первые 20 строк набора данных.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Для столбца total_income посчитаем медиану по зарплате. 

In [11]:
total_income_median = data['total_income'].median() #Посчитаем медиану. 

Заменим методом fillna() пропущенные значения в столбце total_income на медиану по зарплате. 

In [12]:
data['total_income'] = data['total_income'].fillna(value=total_income_median) #Заменим пропущенные значения. 

Выведем первые 20 строк таблицы на экран.

In [13]:
data.head(20) #Выведим на экран первые 20 строк набора данных.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверяем отсутствие количества NaN элементов в записях

In [14]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

В столбцах  days_employed и total_income обнаружено по 2174 NaN объектов, причём они находятся в одних и тех же строках. В остальных стобцах объекты значимы - не  NaN.

Для столбца days_employed была проведена замена пропущеных значений. Пропущенные значения заполнялись средним значением по столбцу, т.к. в рамках нашей задачи данный столбец существенной роли не играет и на него можно особого внимания не обращать. Однако, некорректные данные данного столбца сразу бросаются в глаза: отрицательные значения, стаж указанный в днях превышает возможную длительность жизни человека.  

Для столбца total_income  пропущенные значения заменялись на медианные значения по зарплате данного столбца. Этот метод был выбран, потому что сумма абсолютных (по модулю) отклонений от медианного значения дает минимально возможное значение, если сравнивать с отклонениями от любой другой величины. Следовательно минимально повлияет на искажение исходных данных.

### Замена типа данных
<a id='null'></a>

Проведем замену вещественного типа данных в 'days_employed' и  'total_income'  на целочисленный тип = int32, т.к. числа в 'days_employed' и  'total_income' находятся в диапазоне от -2147483648 по 2147483647. 

In [15]:
data['days_employed'] = data['days_employed'].astype('int32') # замена вещественного типа данных на целочисленный 

In [16]:
data['total_income'] = data['total_income'].astype('int32') # замена вещественного типа данных на целочисленный 

В 'children',  'dob_years', 'education_id', 'family_status_id'  и  'debt' - заменим на int8,  т.к. числа в диапазоне от -128 по 127.

In [17]:
data['children'] = data['children'].astype('int8') 

In [18]:
data['dob_years'] = data['dob_years'].astype('int8') 

In [19]:
data['education_id'] = data['education_id'].astype('int8') 

In [20]:
data['family_status_id'] = data['family_status_id'].astype('int8') 

In [21]:
data['debt'] = data['debt'].astype('int8') 

Выявление специальным методом unique() в столбце 'gender' списка уникальных значений.

In [22]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [23]:
data.info() # получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int32
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int8
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: int32(2), int8(5), object(5)
memory usage: 1.1+ MB


**Вывод**

Метод astype() позволяет нам четко указать тип, который мы хотим преобразовать. Переходим от одного типа данных к другому, передавая параметр внутри метода astype(). 

В данных 'gender' выявлено уникальное значение - XNA. Из представленных данных узнать пол человека или сделать предположение нет возможности, в расчетах данный параметр использовать не будет, поэтому оставляем его как есть.

Перейдя на другой подтип данных, оптимизировали занимаемую память на 45%.

### Обработка дубликатов
<a id='duplicated'></a>

Переведем значения уровня образования в нижний регистр методом str.lower(). Сохраним их в том же столбце 'education'.

In [24]:
data['education'] = data['education'].str.lower() # сохраняем список  в нижнем регистре

Используя стандартный метод duplicated() посчитаем суммарное количество дубликатов по датафрейму.

In [25]:
data.duplicated().sum()# получение суммарного количества дубликатов в таблице data

71

Посмотрим на дубликаты, для определения причин их возникновения.

In [26]:
data[data.duplicated(keep=False)].sort_values(by= ['days_employed', 'total_income', 'dob_years', 'education'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8853,1,63046,23,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
15892,0,63046,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
19321,0,63046,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
20297,1,63046,23,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
3452,0,63046,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,63046,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
9528,0,63046,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
6537,0,63046,71,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
7938,0,63046,71,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы


Удаление дубликатов - drop_duplicates(inplace = True) из data

In [27]:
data.drop_duplicates(inplace = True)# удаление всех дубликатов из таблицы data специальным методом

Проверим все ли дубликаты удалились.

In [28]:
data.duplicated().sum()# проверка на отсутствие дубликатов в таблице data

0

**Вывод**

В данных методом duplicates() обнаружено 54 дубликатa, воспользовавших специальным методом drop_duplicates все дубликаты были удалены (предварительно посмотрев на них). 

Дубликаты могли возникнуть при неправильном заполнении таблицы.

**Вывод**
    
В данных методом duplicates() обнаружено 71 дубликатa, воспользовавших специальным методом drop_duplicates все дубликаты были удалены (предварительно посмотрев на них). 

Дубликаты могли возникнуть при экспорте данных и из-за человеческого фактора.    

### Лемматизация
<a id='lemma'></a>

In [29]:
m = Mystem()
#data['purpose'].value_counts()#сколько раз встречается каждый тип обращений
#data['purpose'].nunique()# количество уникальных значений для  столбца purpose

list_lemmas = [] #создаем список
for i in  data['purpose']: #пишем цикл
    lemma = m.lemmatize(i) #лемматизируем
    list_lemmas.extend(lemma) # вставляем в конец исходного списка все значение итерируемого_объекта
    
nunique_lemmas = Counter(list_lemmas) # подсчитываем числа их упоминаний в тексте
sorted(nunique_lemmas.items(), key = lambda x: x[1], reverse=True) #возвращает новый отсортированный список итерируемого объекта

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

**Вывод**

Из полученного списка интерес для дальнейшего исследования представляют часто повторяющиеся слова: недвижимость, жилье, автомобиль, образование, свадьба.

### Категоризация данных
<a id='start'></a>

In [30]:
def purpose_lemmas(list_lemmas): #создаем новую функцию purpose_lemmas(list_lemmas), которая возвращает: 
    if 'недвижим' in list_lemmas:
                return 'недвижимость' # 'недвижимость', из слов в которых есть часть слова 'недвижим'
    if  'жил' in list_lemmas:
                return 'жилье' # 'жилье', из слов в которых есть часть слова 'жил'
    if 'авто' in list_lemmas:
                return 'автомобиль' # 'автомобиль', из слов в которых есть часть слова 'авто'
    if 'образ' in list_lemmas:
                return 'образование' #  'образование', из слов в которых есть часть слова 'образ'
    if 'свад' in list_lemmas:
                return 'свадьба' # 'свадьба', из слов в которых есть часть слова 'свад'
    return 'другое'    # 'другое', на все остальные слова которые не попали в выборку


data['purpose_lemmas'] = data['purpose'].apply(purpose_lemmas) #добавляем в исходную таблицу новый столбец purpose_lemmas

def income_count(total_income): #создаем новую функцию income_count(total_income), которая возвращает: #    if total_income < 30000: 
    if total_income < 30000:            
        return 'низкая' #  'низкая', если зарплата меньше 30 000 руб.
    if 30000 <= total_income < 60000:
                return 'средняя' # 'средняя', если зарплата находится в диапозоне от 30 000 до 60 000
    if 60000<= total_income < 90000:
                return 'выше-среднего' # 'выше-среднего', если зарплата находится в диапозоне от 60 000 до 90 000
    if 90000<= total_income < 150000:
                return 'высокая' # 'высокая', если зарплата находится в диапозоне от 90 000 до 150 000    
    return 'сверхдоходы'    #  'сверхдоходы', если зарплата находится выше 200 000
#Разделение уровеня жизни клиентов в зависимости от доходов на следующие категории было сделано на основании данных Федеральной службы государственной статистики 
data['income_count'] = data['total_income'].apply(income_count) #добавляем в исходную таблицу новый столбец 'income_count'


**Вывод**

Для ответов на поставленные вопросы в рамках проекта, необходимо категоризировать данные: по целям получения кредита, по заработной плате.

## Шаг 3. Ответьте на вопросы
<a id='qusions'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
data.loc[(data.children < 0), 'children'] = 1 #заменили отрицательные значения на значение по модулю
data.loc[(data.children == 20), 'children'] = 2 #заменили   значения равные 20 на 2

In [32]:
def quotient(value):  #создаем новую функцию  quotient(value), 
                                      #которая возвращает результат деления суммы невозвращенных кредитов к их колличеству отсортированных по стобцу: 
    return print( 'Отношение невозвращенного кредита к', (data.groupby(value)['debt'].sum() / data.groupby(value)['debt'].count()).sort_values(ascending = False))

#pd.pivot_table(data, index='children', values='debt') #Другой вариант
 
quotient('children') # вызываем функцию quotient, в качестве аргумента передаем 'children', по которому будем группировать

Отношение невозвращенного кредита к children
4    0.097561
2    0.094925
1    0.091658
3    0.081818
0    0.075438
5    0.000000
Name: debt, dtype: float64


**Вывод**

Проверяем столбец на корректные данные.

В графе children выявлены некорректные данные. Считаем максимальное и минимальное количество детей - выбросами. Отричательное число необходимо заменить положительным, а 20 - заменить 2, т.к. ноль скорее всего описка. С увеличением количества детей в семье, количество таких семей уменьшается.

Анализ показал, что с увеличением количества детей в семьях растет вероятность задержки по кредиту, хотя в семьях с пятью детьми задолжностей не обнаружено. Лучше всего отдают кредиты бездетные.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
data['family_status'].value_counts() #Выведем статистику по семейному положению методом value_counts()

quotient('family_status')  # вызываем функцию quotient, в качестве аргумента передаем 'family_status' , по которому будем группировать

Отношение невозвращенного кредита к family_status
Не женат / не замужем    0.097509
гражданский брак         0.093471
женат / замужем          0.075452
в разводе                0.071130
вдовец / вдова           0.065693
Name: debt, dtype: float64


**Вывод**

Наименьшую задолжность по кредиту имеют люди похоронившие супруга/супругу. Хуже всего отдают кредиты  не женатые / не замужние или состоящие в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
quotient('income_count')  # вызываем функцию quotient, в качестве аргумента передаем 'income_count' , по которому будем группировать

Отношение невозвращенного кредита к income_count
низкая           0.090909
высокая          0.084398
выше-среднего    0.083792
сверхдоходы      0.079051
средняя          0.059949
Name: debt, dtype: float64


**Вывод**

Клиенты с средним уровнем  дохода (зарплата от 30000 до 60000 тыс. руб.) имеют более низкий показатель по задержке оплаты кредита, чем люди с низким уровнем дохода. 

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
quotient('purpose_lemmas') # вызываем функцию quotient, в качестве аргумента передаем 'purpose_lemmas' , по которому будем группировать

Отношение невозвращенного кредита к purpose_lemmas
автомобиль      0.093590
образование     0.092200
свадьба         0.080034
недвижимость    0.074634
жилье           0.069058
Name: debt, dtype: float64


**Вывод**

Взявшие кредит для операций с жильем и недвижимостью меньше нарушают график выплат. Кредиты, выданные на образование и автомобиль, часто имеют просрочку.

## Шаг 4. Общий вывод
<a id='output'></a>

Про анализировав качественные и количественные зависимости представленных  показателей оценки влияния семейного положения и количества детей клиента на факт погашения кредита в срок. Можно сделать  вывод, что на погашение кредита без задолжностей оказывают влияние такие факторы, как семейное положение, количество детей в семье, уровень дохода и цель получения кредита.

Семьи с пятью детьми не имеют задолжностей по выплате кредита, в то время, как семьи с четырьмя детьми имеют наибольшее количество неплатежей. Нет явной закономерности между количеством детей в семьях и задержкой в выплате по кредиту.
    
Увеличение дохода не способствует уменьшению задолжностей по оплате кредита. При доходах выше-среднего и высокий остается большая вероятность появления долгов по займу.
    
Резюмируя выше сказанное надежного / ненадежного заёмщика можно описать, следующим образом:

| Заёмщик        | количество детей           | семейное положение  | доход        | цель кредита      | 
|:------------------:|:----------------------------------:|: ------------------------------:|:---------------:|:-----------------------:|
| надежный     | без детей | одинокий, ранее состоявший в браке| от 30 000 до 60 000     | жилье / недвижимость  | 
| ненадежный   | 2 или 4      |   Не женат / не замужем  | < 30 000   |  автомобиль / образование   | 


## Чек-лист готовности проекта
<a id='list'></a>

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.